In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier
from fractions import Fraction
import pickle
import random

In [74]:
# Import data
dfWin = pd.read_csv("./DataFrames/WinnersProcessed.csv", index_col=0).dropna(subset=["expected_dmg"]).reset_index(drop=True)
dfLose = pd.read_csv("./DataFrames/LosersProcessed.csv", index_col=0).dropna(subset=["expected_dmg"]).reset_index(drop=True)
dfWin.head()

,name,desc,cr,immunities,resists,conditions,wins,actions,hp,str,dex,con,int,wis,cha,ac,spd,winlen,expected_dmg
0,Wraith of Ogre Power,Medium Undead Neutral Evil,5.0,"Necrotic,Poison,Charmed,Exhaustion,Grappled,Pa...","Acid,Cold,Fire,Lightning,Thunder,Bludgeoning,P...",Gauntlets of Ogre Power,"Werebear, Air Elemental, Troll, Roper, CR 5","Life Drain: +11, 4d8+9 Necrotic & Life Drain",39,19,16,16,12,14,15,13,60,5,18.396
1,Oni,Large Giant Lawful Evil,7.0,NaN,NaN,"Death Defying,Regeneration",NaN,"Glaive: +6, 10ft, 2d10+4 Slashing\tMultiattack...",55,19,11,16,14,12,15,16,30,0,7.961
2,Giant Ape,Huge Beast Unaligned,7.0,NaN,NaN,Shielded,NaN,"Fist: +8, 10ft, 3d10+6 Bludgeoning\tRock: +8, ...",69,23,14,18,7,12,7,12,40,0,12.907
3,Young Brass Dragon,Large Dragon Chaotic Good,6.0,Fire,NaN,Reliable Damage,NaN,"Multiattack: 10ft, Bite, 2 Claw\tBite: +6, 10f...",61,19,10,17,12,11,15,17,80,0,35.000
4,Fire Elemental,Large Elemental Neutral,5.0,"Fire,Poison,Exhaustion,Grappled,Paralyzed,Petr...","Bludgeoning,Piercing,Slashing","+2 CHA,Heated Body",NaN,"Touch: +5, 2d6+3 Fire, On Fire\tMultiattack: F...",46,10,17,16,6,10,9,13,50,0,14.116


I forgot to put which side the winner and loser were on, therefore I need to randomize where to put the winning creature and where to put the losing creature.
This will allow the model from not constantly predicting the creature on a specific side

In [68]:
import random
# Returns all stat values of a dataframe as a 2d array
def getStatsValues(df: pd.DataFrame):
    return df.loc[:, ["cr", "hp", "str", "dex", "con", "int", "wis", "cha", "ac", "spd", "expected_dmg"]].values

def convertToFloat(val):
    if type(val) == str:
        return float(Fraction(val))
    else:
        return float(val)

dfWin["cr"] = dfWin.apply(lambda row: convertToFloat(row["cr"]), axis=1)
dfLose["cr"] = dfLose.apply(lambda row: convertToFloat(row["cr"]), axis=1)
# Get Input values
dfWinStats = getStatsValues(dfWin)
dfLoseStats = getStatsValues(dfLose)
# Append input values together
dfStats = np.concatenate([dfWinStats, dfLoseStats], axis=1)
# Generate random list representing which side won and which side lost

# Make the right side outputs equal 1
rightSide = np.ones(dfStats.shape[0]//2)
# Make left side be zero
leftSide = np.zeros(dfStats.shape[0]//2)
# Append all values to targets
targets = np.append(leftSide, rightSide)
# Shuffle results
random.shuffle(targets)


data = []

for i in range(dfWinStats.shape[0] - 1):
    if targets[i]:
        res = np.concatenate([dfLoseStats[i], dfWinStats[i]])
    else:
        res = np.concatenate([dfWinStats[i], dfLoseStats[i]])
    data.append(res)

In [78]:
def convertToFloat(val):
    if type(val) == str:
        return float(Fraction(val))
    else:
        return float(val)

def getDifData(df1: pd.DataFrame, df2: pd.DataFrame, targets: list, stats=['ac', 'cr', 'spd', 'hp', 'str', 'dex', 'con', 'int', 'wis', 'cha', 'expected_dmg']):
    vals = np.zeros((df1.shape[0], len(stats)))
    for (i, r1), (_, r2) in zip(df1.iterrows(), df2.iterrows()):
        for j, stat in enumerate(stats):
            # If a stat is greater than a believable amount make the stat equal to its opponent
            # This allows for better generalization of the data
            if r1[stat] >= 900: r1[stat] = r2[stat]
            if r2[stat] >= 900: r2[stat] = r1[stat]
            vals[i, j] = r1[stat] - r2[stat] if targets[i] else r2[stat] - r1[stat]
    return vals


rightSize = dfWin.shape[0]//2
leftSize = dfWin.shape[0] - rightSize
# Make the right side outputs equal 1
rightSide = np.ones(rightSize)
# Make left side be zero
leftSide = np.zeros(leftSize)
# Append all values to targets
targets = np.append(leftSide, rightSide)
# Shuffle results
random.shuffle(targets)


dfWin["cr"] = dfWin.apply(lambda row: convertToFloat(row["cr"]), axis=1)
dfLose["cr"] = dfLose.apply(lambda row: convertToFloat(row["cr"]), axis=1)

data = getDifData(dfWin, dfLose, targets)
for row in data:
    print(row)

[  2.       1.     -30.     -24.      -2.      -1.       2.      -4.
  -1.      -7.     -14.9945]
[  1.      1.      0.    -15.      9.     -4.      0.      2.     -1.
   0.     -0.185]
[  5.       0.     -10.      -6.      -5.      -6.       0.       0.
  -2.      -4.      -3.7705]
[ 2.     -2.     40.     24.      1.     -7.      3.     -4.     -4.
 -1.     -1.2474]
[  0.       -3.        0.      -34.      -15.        7.       -3.
   4.       -2.        0.        5.15025]
[ -6.           0.          10.          -7.           8.
  -7.           3.         -10.          -3.          -6.
  -0.37333333]
[ 3.   -1.    0.   18.   -3.    5.    2.   -1.    2.   -3.    1.74]
[ -6.           1.         -40.          -8.          -6.
  -3.          -1.         -12.         -10.          -9.
   6.00216667]
[  4.      1.     20.    -38.      0.     -6.      1.      0.     -3.
   0.      2.805]
[ -2.           1.         -20.           6.           4.
   3.          -1.           4.          -2. 

In [ ]:
# Find better ways to normalize results does not work currently

# def normalize(point):
#     if not point:
#         return 0
#     else: return 1/point

# for i in range(data.shape[0]):
#     for j in range(data.shape[1]):
#         data[i,j] = normalize(data[i,j])
# data

In [79]:
# Get the distribution of target values

one = 0
zero = 0

for val in targets:
    if val:
        one += 1
    else:
        zero += 1

print(f"One: {one}")
print(f"Zero: {zero}")

One: 5454
Zero: 5454


In [89]:
splitSize = 0.75

X_train, X_test, y_train, y_test = train_test_split(data, targets, test_size=(1-splitSize), random_state=47)


In [90]:
print(len(X_test[0]))

11


In [99]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

# params = { 'hidden_layer_sizes': [(200, 100, 90), (2000, 500, 80), (2000, 1500, 80), (3000, 1000, 80)]
#             'solver': ['lbfgs']
# }
# TODO: Implement RandomSearch framework
clf = MLPClassifier(hidden_layer_sizes=(2500, 1200, 200), max_iter=10000, activation='tanh', learning_rate='adaptive', learning_rate_init=1e-5, early_stopping=True, verbose=True)

#gs = GridSearchCV(clf)

clf.fit(X_train, y_train)

Iteration 1, loss = 0.66365916
Validation score: 0.621490
Iteration 2, loss = 0.64055625
Validation score: 0.642247
Iteration 3, loss = 0.63761123
Validation score: 0.637363
Iteration 4, loss = 0.63596373
Validation score: 0.631258
Iteration 5, loss = 0.63200516
Validation score: 0.638584
Iteration 6, loss = 0.63197732
Validation score: 0.631258
Iteration 7, loss = 0.63052244
Validation score: 0.639805
Iteration 8, loss = 0.63006485
Validation score: 0.639805
Iteration 9, loss = 0.62914252
Validation score: 0.634921
Iteration 10, loss = 0.62542787
Validation score: 0.632479
Iteration 11, loss = 0.62742296
Validation score: 0.636142
Iteration 12, loss = 0.62395908
Validation score: 0.643468
Iteration 13, loss = 0.62394214
Validation score: 0.648352
Iteration 14, loss = 0.62269003
Validation score: 0.638584
Iteration 15, loss = 0.62154367
Validation score: 0.634921
Iteration 16, loss = 0.62168643
Validation score: 0.638584
Iteration 17, loss = 0.62127473
Validation score: 0.638584
Iterat

MLPClassifier(activation='tanh', early_stopping=True,
              hidden_layer_sizes=(2500, 1200, 200), learning_rate='adaptive',
              learning_rate_init=1e-05, max_iter=10000, verbose=True)

In [100]:
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.63      0.65      0.64      1360
         1.0       0.64      0.62      0.63      1367

    accuracy                           0.64      2727
   macro avg       0.64      0.64      0.64      2727
weighted avg       0.64      0.64      0.64      2727



In [101]:
confusion_matrix(y_test, predictions)

array([[886, 474],
       [516, 851]])

In [ ]:


# save the classifier to a file
with open("./Models/classifier_64perc.pkl", "wb") as file:
    pickle.dump(clf, file)

In [ ]:
# load the saved classifier from the file
with open("./Models/classifier_64perc.pkl", "rb") as file:
    clf = pickle.load(file)

predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))
clf

              precision    recall  f1-score   support

         0.0       0.65      0.55      0.59       677
         1.0       0.60      0.70      0.65       671

    accuracy                           0.62      1348
   macro avg       0.62      0.62      0.62      1348
weighted avg       0.62      0.62      0.62      1348



MLPClassifier(activation='tanh', hidden_layer_sizes=(1000, 1500, 500, 100),
              learning_rate_init=1e-07, max_iter=10000, verbose=True)

In [ ]:
preds = clf.predict(X_train)
print(classification_report(y_train, preds))

              precision    recall  f1-score   support

         0.0       0.61      0.52      0.56      1480
         1.0       0.58      0.67      0.62      1461

    accuracy                           0.60      2941
   macro avg       0.60      0.60      0.59      2941
weighted avg       0.60      0.60      0.59      2941



In [ ]:
clf

MLPClassifier(activation='tanh', hidden_layer_sizes=(1000, 1500, 500, 100),
              learning_rate_init=1e-07, max_iter=10000, verbose=True)